# Setup data for use in the process

In [ ]:
import pandas as pd

data = pd.read_excel("Abstract-Keyword.xlsx")

abstract = []
keyword = []

abstract.extend(data['Abstract'].tolist())
keyword.extend(data['Keyword'].tolist())

# LangGraph, OpenAI API Setup

In [ ]:
import os
import json


with open('config.json') as config_file:
    config = json.load(config_file)


os.environ['LANGCHAIN_TRACING_V2'] = config['LANGCHAIN_TRACING_V2']
os.environ['LANGCHAIN_ENDPOINT'] = config['LANGCHAIN_ENDPOINT']
os.environ['LANGCHAIN_API_KEY'] = config['LANGCHAIN_API_KEY']
os.environ['LANGCHAIN_PROJECT'] = config['LANGCHAIN_PROJECT']
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

# Main Code

### LangGraph GraphState

In [ ]:
from typing import TypedDict

class GraphState(TypedDict):
    abstract : str 
    keyword : str 
    past_agent_generate : str # Attribute definitions from each agent in the previous step
    present_agent_generate : str # Current attribute definitions from each agent
    definability : str # Result of attribute definability check
    moderator_feedback : str # Current feedback from Moderator
    agent_feedback : str # Current feedback from each agent
    meaningless_3_2_count : int # Count of repeated meaningless attribute definitions within the same attribute
    meaningless_3_2_definition : str # Determine whether the process stops due to repeated meaningless attribute definitions during the Debate phase
    debate_state : str # Set current state for progressing to the next step in the Debate phase
    meaningless_4_2_count : int # Count of meaningless attribute definitions
    Final_Keyword_Result : str # Store final definitions of keyword attributes
    status : str # Current Status
    meaning_of_words : str # Store search results for keyword meanings

### keyword attribute classification using multi-agent

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import CommaSeparatedListOutputParser 
import re
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.agents import load_tools, initialize_agent
import time
from collections import defaultdict


MODEL = "gpt-4o"

# Agent Info
Experts_Agent = [
        'Natural Science Expert',
        'Life Science Expert',
        'Civil/Architecture Engineering Expert',
        'Electrical/Electronic Expert',
        'Chemical/Metallurgical Engineering Expert',
        'Healthcare Expert',
        'Education Expert'
    ]

Agent_Info = """
1. Natural Science Expert
 - An expert with research experience in 'physics, astronomy, meteorology, chemistry, geology, and geophysics', capable of understanding the precise meaning and relevance of keywords related to natural sciences and performing practical classification tasks.
2. Life Science Expert
 - An expert with research experience in 'life sciences fields such as biology, botany, zoology, ecology, environmental science, agronomy', capable of understanding the precise meaning and relevance of keywords related to life sciences and performing practical classification tasks.
3. Civil/Architecture Engineering Expert
 - An expert with research experience in 'engineering fields such as industry, architecture, civil engineering, environmental engineering, mechanical engineering, chemical engineering, and mining (minerals)', capable of understanding the precise meaning and relevance of keywords related to engineering and performing practical classification tasks.
4. Electrical/Electronic Expert
 - An expert with research experience in 'electricity, electronics, and telecommunications', capable of understanding the precise meaning and relevance of keywords related to electrical/electronic fields and performing practical classification tasks.
5. Chemical and Metallurgical Engineering Expert
 - An expert with experience in 'chemical processes, petroleum, and metallurgy', capable of understanding the precise meaning and relevance of keywords related to chemical and metallurgical engineering and performing practical classification tasks.
6. Healthcare Expert
 - An expert with experience in 'medical care, pharmacy, and nursing', capable of understanding the precise meaning and relevance of keywords related to healthcare and performing practical classification tasks.
7. Education Expert
 - An expert, such as a university professor, with extensive knowledge and teaching experience in fields such as natural sciences, engineering, and medical sciences. This expert is capable of understanding the precise meaning and relevance of keywords, as well as research trends in these fields, and performing practical classification tasks.
"""

# Attribute Info
Attribute_Info = """
# Attribute Type Info)
1. Materials & Sample
 - Keywords representing key substances, organisms, or compounds used in experiments, tests, or analyses.
 - If the keyword is a "molecular name," it is classified under this attribute unless exceptions apply.
 - Proteins, enzymes, cells, viruses, polymers, and elemental symbols are likely to be classified under this attribute and are defined as such unless special exceptions exist.
2. Research Technique & Equipment
 - Keywords related to experimental, measurement, or analytical methods, numerical ratios, concepts, or specific equipment used in research.
 - If the keyword corresponds to the name of equipment, it is classified under this attribute unless exceptions apply.
3. Properties & Characteristics
 - Keywords representing the physical, chemical, structural, cellular, or behavioral properties of the target material.
 - Most keywords classified under this attribute, such as physical, chemical, or structural properties, have measurable and quantifiable features. Cellular or behavioral properties, such as human introversion/extroversion or animal nocturnality, may be difficult to quantify and require careful classification.
 - This attribute includes the "results" of observation and measurement.
4. Processes & Reactions
 - Keywords related to specific processes or phenomena observed (or utilized) in the study, including chemical reactions, symptoms (disease/symptom), diseases, or scientific reactions.
 - This attribute represents the "phenomena" that are subjects of observation and measurement.
5. Applications & Uses
 - Keywords indicating how the study's results or findings can be utilized or applied.
 - If the abstract includes discussions related to applications, prioritize classifying the keyword under this attribute. If not, analyze whether the keyword can be defined under another attribute and assign it to the most appropriate one.
6. Time & Chronology
 - Keywords representing specific times or chronologies addressed in the study.
 - This attribute includes keywords referring to historical/geological events or phenomena, climate change, or specific time points in the past or future.
7. Topic (*Allows Overlapping Definitions*)
 - Keywords representing the subject matter of the study.
 - For this attribute, overlapping definitions are permitted. Define the primary attribute under another category and add "Topic" as a secondary attribute. Avoid defining "Topic" as a standalone attribute whenever possible.
 - Keywords related to academic disciplines or fields of study should be classified under this attribute unless exceptions apply.
8. Site & Facility & Environment
 - Keywords representing specific regions, countries, locations, or environments where the research or experiment is conducted.
"""

#################################################################################################################################################

# Compare previous and current attribute results
def Compare_Keyword_Attribute(experts, ex_experts):
    compare_attribute = False
    for key in experts:
        if experts[key]['keyword_attribute'].strip() != ex_experts[key]['keyword_attribute'].strip():
            compare_attribute = True
            break
    return compare_attribute


# Split data by individual expert
def Sep_Agent(present_agent_generate):
    err = re.search(r'\d', present_agent_generate)

    if err:
        present_agent_generate = present_agent_generate[err.start():]

    if present_agent_generate.startswith('"'):
        present_agent_generate = present_agent_generate[1:]  
    
    sections = present_agent_generate.strip().split('\n\n')


    experts = {}
    re_experts = {}
    experts_str = ""
    index = 1

    for section in sections:
        lines = section.split('\n')
        lines = [line.strip() for line in lines if line.strip()]
        # Extract keywords
        expert_name = lines[0].split('. ')[1] if '. ' in lines[0] else lines[0]

        if expert_name.startswith('"'):
            expert_name = expert_name[1:]
            
        # Extract attribute
        keyword_attribute_line = next(line for line in lines if line.startswith('Keyword Attribute'))
        keyword_attribute = keyword_attribute_line.split(':')[1].strip()
        
        # Extract justification for definitions
        definition_reason_line = next(line for line in lines if line.startswith('Justification'))
        definition_reason = definition_reason_line.split(':')[1].strip()

        experts[expert_name] = {
            'keyword_attribute': keyword_attribute,
            'definition_reason': definition_reason
        }

        # Exclude cases where attribute value is '-'
        if keyword_attribute != '-':
            re_experts[expert_name] = {
                'keyword_attribute': keyword_attribute,
                'definition_reason': definition_reason
            }

    
    for expert_name, expert_info in experts.items():
        experts_str += f"{index}. {expert_name}\n"
        experts_str += f"Keyword Attribute : {expert_info['keyword_attribute']}\n"
        experts_str += f"Justification : {expert_info['definition_reason']}\n\n"
        index += 1


    experts_str = experts_str.strip()
    
    return experts, re_experts, experts_str




# Count total attribute definitions for each agent
def Check_Keyword_Attribute_Count(experts):
    topic_exists = any(', Topic' in value['keyword_attribute'] or 'Topic, ' in value['keyword_attribute'] for value in experts.values())

    if topic_exists: #When Topic and other attributes exist, check if only one non-Topic attribute is defined
        unique_attributes = set()
        
        for key, value in experts.items():
            attributes = value['keyword_attribute'].split(', ')
            filtered_attributes = [attr for attr in attributes if attr != 'Topic' and attr != '-']
            unique_attributes.update(filtered_attributes)
    else: # When Topic is the only attribute, check if each agent has exactly one attribute including Topic
        # If Topic does not exist, print all keyword_attribute values and their counts
        unique_attributes = set()
        
        for value in experts.values():
            attributes = value['keyword_attribute'].split(', ')
            filtered_attributes = [attr for attr in attributes if attr != '-']
            unique_attributes.update(filtered_attributes)
    
    final_attribute_count = len(unique_attributes)
    
    return final_attribute_count




#  Modify format for agent-specific attribute definitions and their justifications to be used in the feedback phase
def Group_Experts_By_Non_Topic(re_experts):
    attribute_groups = defaultdict(list)

    
    for expert_name, info in re_experts.items():
        # Extract attributes excluding Topic
        keyword_attributes = [
            attr.strip() for attr in info['keyword_attribute'].split(', ')
            if attr != 'Topic'
        ]
        
        for attribute in keyword_attributes:
            definition_reason = info['definition_reason']
            attribute_groups[attribute].append(f"Reason for {expert_name}'s Attribute Definition: \n{definition_reason}")
    
    result_str = ""
    

    for keyword_attribute, definitions in attribute_groups.items():
        result_str += f"[[[{keyword_attribute}]]]\nSupporting Reasons from Agents Who Defined This Attribute))\n"
        result_str += "\n\n".join(definitions)  
        result_str += "\n\n\n"  

    return result_str.strip()


# user_prompt_1 : brief description of the task / user_prompt_2 : task guidelines / keyword_info : information required for the task
def ChatModel_Generation(system_prompt, user_prompt_1, keyword_info, user_prompt_2=""):
    if user_prompt_2:
        user_prompt = user_prompt_1 + Agent_Info + "\n\n" + Attribute_Info + "\n\n" + user_prompt_2 + "\n\n" + keyword_info
    else: # For word meaning search
        user_prompt = user_prompt_1 + "\n\n" + keyword_info

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", user_prompt),
    ])

    model = ChatOpenAI(model=MODEL)

    chain = prompt | model
    generated_text = chain.invoke({})

    return generated_text.content

    



#################################################################################################################################################




# 0. Search for Keyword Meaning
def Meaning_Of_Keyword(keyword):
    meaning_of_keyword_prompt = """You are a helpful assistant. Your role is to provide users with etymological analysis, semantic analysis, and overall keyword meaning for a given keyword."""

    meaning_of_keyword_content = f"""You possess extensive knowledge in the field of science. Using the keywords provided by the user, identify the meaning of the keywords by first discussing their etymological meanings and individual word meanings. Then, provide an analysis of the overall meaning of the complete keyword. All responses should be generated in English."""

    keyword_info = f"""# Keyword Requiring Meaning Analysis))
{keyword}"""

    gen_result = ChatModel_Generation(meaning_of_keyword_prompt, meaning_of_keyword_content, keyword_info)

    return gen_result


def N_Meaning_Of_Keyword(state : GraphState) -> GraphState:
    Keyword = state["keyword"]
            
    keyword_mean = Meaning_Of_Keyword(Keyword)
    
    keyword_result = f'{Keyword}: {keyword_mean}'

    return GraphState(meaning_of_words = keyword_result)


# 1. Initial Attribute Definition
def First_Same_Agent_Generate(abstract, keyword, my_agent, meaning_words):
    same_filed_prompt = """You are an expert in your field, possessing specialized knowledge. Your role is to assume the persona of an expert Agent within your specific domain and utilize your expertise to classify keywords based on the provided abstract of a research paper. In other words, your task is to define the attributes of the keywords according to the given area of expertise. You will receive information about the Agent and the attributes, and then you will be provided with the abstract of the research paper and the keywords from the user. All responses should be in English."""

    same_filed_content = """Review the following Agent and attribute information, and adhere to the provided guidelines. Assume the role of the assigned Agent and utilize your expertise in the relevant field to refer to the given abstract and define the most appropriate attribute for the keyword.
    
# Agent Info)"""

    same_filed_content_2 = """
# Agent Setting and Attribute Definition Guidline
1. Assume the role of the assigned Agent from the seven provided, utilizing expertise in the relevant field to define attributes.
2. If the keyword is unrelated to the assigned Agent’s field, do not define an attribute. Instead, mark it as "-" to indicate irrelevance.
3. When defining an attribute, provide both the classification and a justification. The reasoning must be based on the keyword’s inherent meaning and the abstract, focusing on why the keyword should be assigned to the chosen attribute. The justification should be as detailed as possible, ensuring logical validity and supporting the classification with strong evidence.
4. Agents must prioritize the inherent meaning of the keyword when defining attributes. However, if the abstract indicates that the keyword is used in a different context than its original meaning, this should be taken into account, and the most appropriate attribute should be assigned accordingly.
5. Agent should classify keyword attributes based on their expertise in the assigned field, but not by considering how the keyword is used or perceived within that field. In other words, attribute classification should focus on the inherent meaning of the keyword itself, rather than its value or application in a specific domain.
6. <Important> The primary focus should be on the inherent meaning of the keyword. However, if the abstract reveals that the keyword is used in a different context from its original meaning, attribute classification should not be based on its association with a specific field ("keyword-field" relationship). Instead, classification must rely on how the keyword is used and interpreted within the abstract ("keyword-abstract" relationship). Therefore, in such cases, the Agent must utilize their domain expertise to analyze how the keyword is described and applied in the abstract. The reasoning for why the keyword should be assigned to a specific attribute must be thoroughly explained and logically justified in the attribute definition.
7. Each of the 1-7 Agents represents a broad category, so sub-field experts should be assigned within each Agent. These sub-field experts must engage in discussions to determine the most appropriate attribute classification, which will then be established as the final attribute classification for that Agent. For example, in the case of the '1. Natural Science Expert' Agent, experts in physics, astronomy, meteorology, chemistry, geology, and geophysics possess extensive knowledge in their respective domains. Therefore, after discussions among these specialists, the most suitable attribute classification should be determined and set as the final classification for the '1. Natural Science Expert' Agent.
8. For the '7. Topic' attribute, a keyword can be classified under this category when it represents the main subject of the study. Since such keywords can also possess other relevant attributes, they are not limited to the 'Topic' classification alone. Therefore, it is important to note that 'Topic' is the only attribute that can be defined alongside another attribute. 
9. <Important> Except for "7. Topic," no attribute should have overlapping definitions. Only the most suitable attribute should be selected.
10. When classifying attributes, note that observed and examined "phenomena" fall under the Processes & Reactions attribute, whereas observed and measured "results" are classified under Properties & Characteristics. For example, terms such as "Boiling," "Evaporation," "Molecular binding," and "Thermal expansion" belong to Processes & Reactions, while "boiling point," "vapor pressure," "binding energy," and "coefficient of thermal expansion" fall under Properties & Characteristics. Carefully analyze these characteristics when determining the appropriate attribute for a given keyword.
11. Similar keywords may have different attributes depending on their prefixes and suffixes. Therefore, consider both the prefix, suffix, and overall meaning of the keyword when defining its attribute.
12. The generated output must strictly follow the structure below. No unnecessary content should be included beyond this format, and responses must adhere precisely to the specified structure. (Insert the assigned Agent name in "OOO" and replace "~~~" with the appropriate values.)
Generation Structure) OOO Expert\nKeyword Attribute: ~~~\nJustification: ~~~
"""

    keyword_info = f"""# Keyword Meaning(Fully understand the word’s meaning before proceeding with attribute classification))
{meaning_words}

# User-Provided Keyword and Related Abstract Info)
- Keyword requiring attribute classification: {keyword}
- Relevant abstract information from the related paper:
{abstract}

- Assigned Agent for Attribute Classification: {my_agent}
"""

    gen_result = ChatModel_Generation(same_filed_prompt, same_filed_content, keyword_info, same_filed_content_2)

    return gen_result


def N_All_First_Same_Agent_Generate(state : GraphState) -> GraphState:
    Abstract = state["abstract"]
    Keyword = state["keyword"]
    Meaning_of_words = state["meaning_of_words"]

    err_count = 0

    while True:
        try:
            Result = []
            
            
            for i, all_experts in enumerate(Experts_Agent, start = 1):
                generate = First_Same_Agent_Generate(Abstract, Keyword, all_experts, Meaning_of_words)
        
                num_result = f'{i}. {generate}'
                Result.append(num_result)
        
            fsag = '\n\n'.join(Result)

            if err_count > 6:
                break

            test_ex_experts, test_ex_re_experts, test_ex_experts_str = Sep_Agent(fsag)
            break
        except Exception as e:
            print("Sep_Agent error detected → Regenerating")
            err_count += 1
            continue

    return GraphState(present_agent_generate = fsag, meaningless_3_2_count = 0, meaningless_4_2_count = 0)

            


# 2-1. Raise questions to each agent and determine if attributes can be defined (Moderator)
def All_Same_Keyword_Master(abstract, keyword, present_agent_generate, meaning_words):
    all_same_keyword_master_prompt = """As an expert in classifying keyword attributes, you are the Moderator of this Debate process. You have a good understanding of keywords and attributes, related knowledge, and classification of keyword attributes in the Debate process. Your role is to analyze the attributes and reasons of keywords defined by experts in each field and to present questions by pointing out whether the attribute definition made by each agent represents the keyword well and the agent's errors. Agent information, attribute information, and attribute definition information for each agent are given, and abstract of the paper and keyword information are received from the user. All answers should be in English."""

    all_same_keyword_master_content = """Review the Agent and attribute information below, ensuring compliance with the guidelines. Verify whether the attribute values defined by each Agent accurately represent the keyword, and follow the instructions provided.

# Info on Agents Participating in the Debate)"""

    all_same_keyword_master_content_2 = """
# Instructions
1. Determine whether the attribute values defined by each of the seven Agents are suitable for the keyword and mark the result as O/ X.
2. In the attribute definitions and justifications for each Agent, a '-' is used when an Agent is unable to define an attribute for a keyword that is unrelated to their field.
3. If all attribute values defined by the Agents are appropriate for the keyword, there is no need to raise any questions. In this case, the question field should be marked with '-'.
4. Since the keyword attributes are defined through this process, when analyzing each Agent, carefully identify and evaluate any weaknesses in the attribute definitions with a strict and objective perspective.
5. If any of the seven Agents need to revise their attribute definition, raise questions and provide feedback on aspects they may have overlooked, such as specific conditions or concepts, to guide them toward a more accurate attribute definition.
6. Also, Carefully review the attribute values and justifications provided by each Agent. Identify any errors in the keyword attribute definition, and if the justification lacks sufficient reasoning, request additional evidence or empirical support. Ensure that the attribute is defined correctly based on valid reasoning.
7. If a more meaningful attribute value can be derived than the one defined by the Agent, present it as a question for further consideration.
8. If there are no significant errors in the attribute values defined by each Agent and they are deemed acceptable, provide comments summarizing key considerations or cautionary points that may assist in the final attribute definition.
9. When analyzing the attribute definitions provided by each Agent, ensure that they are not overinterpreting the abstract and that the attribute assignment is primarily based on the keyword's inherent meaning, with the abstract serving as supplementary context. The reasoning behind why the keyword is assigned to a particular attribute should be carefully examined for validity, and any concerns or inconsistencies should be raised accordingly.
10. <Important> The primary focus should be on the inherent meaning of the keyword. However, if the abstract reveals that the keyword is used in a different context from its original meaning, attribute classification should not be based on its association with a specific field ("keyword-field" relationship). Instead, classification must rely on how the keyword is used and interpreted within the abstract ("keyword-abstract" relationship). Therefore, in such cases, the Agent must utilize their domain expertise to analyze how the keyword is described and applied in the abstract. The reasoning for why the keyword should be assigned to a specific attribute must be thoroughly explained and logically justified in the attribute definition. With this in mind, support each Agent in basing their keyword classification on factual evidence, identifying solid justifications, and providing well-reasoned explanations for their attribute assignments.
11. For the '7. Topic' attribute, a keyword can be classified under this category when it represents the main subject of the study. Since such keywords can also possess other relevant attributes, they are not limited to the 'Topic' classification alone. Therefore, it is important to note that 'Topic' is the only attribute that can be defined alongside another attribute.
12. <Important> Except for "7. Topic," no attribute should have overlapping definitions. Only the most suitable attribute should be selected.
13. When classifying attributes, note that observed and examined "phenomena" fall under the Processes & Reactions attribute, whereas observed and measured "results" are classified under Properties & Characteristics. For example, terms such as "Boiling," "Evaporation," "Molecular binding," and "Thermal expansion" belong to Processes & Reactions, while "boiling point," "vapor pressure," "binding energy," and "coefficient of thermal expansion" fall under Properties & Characteristics. Carefully analyze these characteristics when determining the appropriate attribute for a given keyword.
14. When defining attributes, Agents should consider how the keyword is used within the given paper. However, the attribute classification must ultimately be based on the fundamental meaning of the keyword itself.
15. Agent should classify keyword attributes based on their expertise in the assigned field, but not by considering how the keyword is used or perceived within that field. In other words, attribute classification should focus on the inherent meaning of the keyword itself, rather than its value or application in a specific domain.
16. Similar keywords may have different attributes depending on their prefixes and suffixes. Therefore, consider both the prefix, suffix, and overall meaning of the keyword when defining its attribute.
17. When presenting questions and determining the validity of each Agent's attribute definition, carefully review their assigned attribute value and justification. Ensure that the Agent has not reached an incorrect conclusion by including false or misleading information and that their classification is based on factual evidence.
18. <Important> If, after reviewing all the justifications for the currently defined attribute values, there is a possibility that a more suitable attribute could be assigned, it is essential to present that attribute along with appropriate reasoning. This allows for highlighting the fact that, in addition to the proposed attributes, the newly suggested attribute is also a valid classification for the keyword.
19. <Important> If either an individual question or a general questions exists, the final keyword attribute classification should be marked as 'not possible'. Additionally, if the general questions includes corrective comments or negative feedback indicating the need for revision, the attribute classification should also be set to 'not possible'.
20. Attribute classification must strictly define the primary attribute of the keyword. Do not provide auxiliary or secondary attributes.
21. The generated output must strictly follow the structure below. Do not include any unnecessary content beyond this format. (The information inside parentheses serves as a guideline and should be replaced with the corresponding content when writing.)
Generation Structure)
General Questions)
(Write question and feedback that apply universally, regardless of the specific Agent)

Agent-Specific Questions)
1. Natural Science Expert 
Attribute Definability : O/X (Choose one)
Question : 

2. Life Science Expert
Attribute Definability :  O/X (Choose one)
Question : 

3. Civil/Architecture Engineering Expert
Attribute Definability :  O/X (Choose one)
Question : 

.
.
(Generate the same structure for the remaining Expert Agent as well)


###
Final Keyword Attribute Definability : possible/not possible (Choose one)
"""

    keyword_info = f"""Keyword Meaning (Fully understand the word’s meaning before proceeding with attribute classification))
{meaning_words}

# User-Provided Keyword and Related Abstract Info)
- Keyword requiring attribute classification: {keyword}
- Relevant abstract information from the related paper:
{abstract}


# Attribute Definitions and Justifications for Each Agent)
{present_agent_generate}
"""

    gen_result = ChatModel_Generation(all_same_keyword_master_prompt, all_same_keyword_master_content, keyword_info, all_same_keyword_master_content_2)

    return gen_result

def N_All_Same_Keyword_Master(state : GraphState) -> GraphState:
    Abstract = state["abstract"]
    Keyword = state["keyword"]
    present_agent_generate = state["present_agent_generate"]
    meaningless_3_2_count = state["meaningless_3_2_count"]
    Meaning_of_words = state["meaning_of_words"]
    
    
    generate = All_Same_Keyword_Master(Abstract, Keyword, present_agent_generate, Meaning_of_words)

    split_generate = generate.split('Final Keyword Attribute Definability : ')

    definability = split_generate[1].strip() if len(split_generate) > 1 else ""
    moderator_feedback = split_generate[0].strip()

    if ("possible" not in definability.strip().lower()) and ("not possible" not in definability.strip().lower()):
        definability = "err"

    if ("possible" in definability.strip().lower()):
        definability = "possible"
    elif ("not possible" in definability.strip().lower()):
        definability = "not possible"

    
        
        

    return GraphState(definability = definability, moderator_feedback = moderator_feedback)




# 3-1. Summarize final attribute definitions and justifications from agents' attribute definition information
def Final_Step_Keyword_Attribute(abstract, keyword, present_agent_generate):
    final_step_prompt = """You are a helpful assistant. Your task is to gather the opinions of each expert and give them appropriate attributes for the 'keywords' they get from the user, and summarize the reasons for defining the keyword attributes by referring to the attribute definitions defined by experts in each field. In other words, you must combine the attribute definitions defined by experts in each field with their expertise in their field, and provide them with a summary of the attribute definitions for the keyword and why. Agent information and attribute information, the attribute definition values for each agent, and the reason information will be given, and the abstract of the thesis and keyword information will be received from the user. All answers should be in English."""

    final_step_content = """Review the Agent and attribute information below, ensuring compliance with the guidelines. Consolidate the attribute values and justifications provided by each Agent to derive the final attribute for the given keyword.

# Agent Info)"""

    final_step_content_2 = """
# Agent Configuration and Attribute Definition Criteria
1. If an Agent's assigned keyword attribute value is '-', it indicates that the keyword is unrelated to their field and should be excluded. Instead, refer to the meaningful attribute values provided by the relevant Agents.
2. Since the final attribute for the keyword must be determined, consolidate the attribute values and justifications provided by each Agent into a clear and structured summary.
3. When defining an attribute, provide both the classification and a justification. The reasoning must be based on the keyword’s inherent meaning and the abstract, focusing on why the keyword should be assigned to the chosen attribute. The justification should be as detailed as possible, ensuring logical validity and supporting the classification with strong evidence.
4. For the '7. Topic' attribute, a keyword can be classified under this category when it represents the main subject of the study. Since such keywords can also possess other relevant attributes, they are not limited to the 'Topic' classification alone. Therefore, it is important to note that 'Topic' is the only attribute that can be defined alongside another attribute.
5. <Important> Except for "7. Topic," no attribute should have overlapping definitions. Only the most suitable attribute should be selected.
6. When classifying attributes, note that observed and examined "phenomena" fall under the Processes & Reactions attribute, whereas observed and measured "results" are classified under Properties & Characteristics. For example, terms such as "Boiling," "Evaporation," "Molecular binding," and "Thermal expansion" belong to Processes & Reactions, while "boiling point," "vapor pressure," "binding energy," and "coefficient of thermal expansion" fall under Properties & Characteristics. Carefully analyze these characteristics when determining the appropriate attribute for a given keyword.
7. Similar keywords may have different attributes depending on their prefixes and suffixes. Therefore, consider both the prefix, suffix, and overall meaning of the keyword when defining its attribute.
8. <Important> The justification must not include the Agent's name or any specific field. Instead, it should present a comprehensive reasoning for the keyword’s attribute classification. When describing the justification, avoid references such as "based on the opinions of all Agents" or "as stated by ?? expert." Focus solely on the general basis for the attribute classification.
9. The generated output must follow the structure "Keyword : Attribute Type - Reason." Do not include any unnecessary content beyond this format.
In other words, the keyword name and its defined attribute value should be separated by a colon (:), and the attribute value and its justification should be separated by a hyphen (-).
"""

    keyword_info = f"""# User-Provided Keyword and Related Abstract Info)
- Keyword requiring attribute classification: {keyword}
- Relevant abstract information from the related paper:
{abstract}

# Attribute Definitions and Justifications for Each Agent)
{present_agent_generate}


"""

    gen_result = ChatModel_Generation(final_step_prompt, final_step_content, keyword_info, final_step_content_2)

    return gen_result

def N_Final_Step_Keyword_Attribute(state : GraphState) -> GraphState:
    Abstract = state["abstract"]
    Keyword = state["keyword"]
    present_agent_generate = state["present_agent_generate"]
    
    
    final_keyword_result = Final_Step_Keyword_Attribute(Abstract, Keyword, present_agent_generate)

    return GraphState(Final_Keyword_Result = final_keyword_result)




# 3-2. Revised attribute definition
def Re_All_Same_Keyword_Master(abstract, keyword, present_agent_generate, master_doubt, my_agent, meaning_words):
    re_all_same_keyword_master_prompt = """You are an expert in your field, possessing specialized knowledge. Your role is to assume the persona of an expert Agent within your specific domain and utilize your expertise to classify keywords based on the provided abstract of a research paper. In other words, your task is to define the attributes of the keywords according to the given area of expertise. You will receive information about the Agent and the attributes, and then you will be provided with the abstract of the research paper and the keywords from the user. All responses should be in English."""

    re_all_same_keyword_master_content = """Review the Agent and attribute information, as well as the previous attribute definition concerns. Adhering to these guidelines, assume the role of the assigned Agent and utilize expertise in the respective field to redefine the keyword with the most appropriate attribute, referring to the provided abstract.

# Agent Info)"""

    re_all_same_keyword_master_content_2 = """
# Agent Configuration and Attribute Definition Criteria
1. Assume the role of the assigned Agent from the seven provided, utilizing expertise in the relevant field to define attributes.
2. <Important> Each Agent must recall their previously defined attribute value and use it as a reference to carry out the following process.
3. Each Agent must review the provided previous attribute definition questions(feedback) and incorporate the feedback when revising their attribute classification.
4. If the keyword is unrelated to the assigned Agent’s field, do not define an attribute. Instead, mark it as "-" to indicate irrelevance.
5. <Important> After revising the attribute definition, explain the updated justification, highlighting any differences from the previous definition and addressing any raised concerns. The reasoning should be grounded in the keyword's meaning and the abstract, clearly demonstrating why the keyword fits the assigned attribute. Provide a thorough and well-supported explanation.
6. Agents must prioritize the inherent meaning of the keyword when defining attributes. However, if the abstract indicates that the keyword is used in a different context than its original meaning, this should be taken into account, and the most appropriate attribute should be assigned accordingly. 
7. <Important> The primary focus should be on the inherent meaning of the keyword. However, if the abstract reveals that the keyword is used in a different context from its original meaning, attribute classification should not be based on its association with a specific field ("keyword-field" relationship). Instead, classification must rely on how the keyword is used and interpreted within the abstract ("keyword-abstract" relationship). Therefore, in such cases, the Agent must utilize their domain expertise to analyze how the keyword is described and applied in the abstract. The reasoning for why the keyword should be assigned to a specific attribute must be thoroughly explained and logically justified in the attribute definition. Keep this in mind when redefining the keyword's attribute and ensure that the justification is appropriately explained.
8. Agent should classify keyword attributes based on their expertise in the assigned field, but not by considering how the keyword is used or perceived within that field. In other words, attribute classification should focus on the inherent meaning of the keyword itself, rather than its value or application in a specific domain.
9. Each of the 1-7 Agents represents a broad category, so sub-field experts should be assigned within each Agent. These sub-field experts must engage in discussions to determine the most appropriate attribute classification, which will then be established as the final attribute classification for that Agent. For example, in the case of the '1. Natural Science Expert' Agent, experts in physics, astronomy, meteorology, chemistry, geology, and geophysics possess extensive knowledge in their respective domains. Therefore, after discussions among these specialists, the most suitable attribute classification should be determined and set as the final classification for the '1. Natural Science Expert' Agent.
10. For the '7. Topic' attribute, a keyword can be classified under this category when it represents the main subject of the study. Since such keywords can also possess other relevant attributes, they are not limited to the 'Topic' classification alone. Therefore, it is important to note that 'Topic' is the only attribute that can be defined alongside another attribute.
11. <Important> Except for "7. Topic," no attribute should have overlapping definitions. Only the most suitable attribute should be selected.
12. When classifying attributes, note that observed and examined "phenomena" fall under the Processes & Reactions attribute, whereas observed and measured "results" are classified under Properties & Characteristics. For example, terms such as "Boiling," "Evaporation," "Molecular binding," and "Thermal expansion" belong to Processes & Reactions, while "boiling point," "vapor pressure," "binding energy," and "coefficient of thermal expansion" fall under Properties & Characteristics. Carefully analyze these characteristics when determining the appropriate attribute for a given keyword.
13. Similar keywords may have different attributes depending on their prefixes and suffixes. Therefore, consider both the prefix, suffix, and overall meaning of the keyword when defining its attribute.
14. The generated output must strictly follow the structure below. No unnecessary content should be included beyond this format, and responses must adhere precisely to the specified structure. (Insert the assigned Agent name in "OOO" and replace "~~~" with the appropriate values.)
Generation Structure) OOO Expert\nKeyword Attribute: ~~~\nJustification: ~~~
"""

    keyword_info = f"""# Keyword Meaning (Fully understand the word’s meaning before proceeding with attribute classification))
{meaning_words}

# User-Provided Keyword and Related Abstract Info)
- Keyword requiring attribute classification: {keyword}
- Relevant abstract information from the related paper:
{abstract}


- Assigned Agent for Attribute Classification: {my_agent}


# Previous Attribute Definitions and Justifications for Each Agent)
{present_agent_generate}

# Relevant Questions(Feedback) to Consider)
{master_doubt}
"""

    gen_result = ChatModel_Generation(re_all_same_keyword_master_prompt, re_all_same_keyword_master_content, keyword_info, re_all_same_keyword_master_content_2)

    return gen_result


def N_Re_All_Same_Keyword_Master(state : GraphState) -> GraphState:
    Abstract = state["abstract"]
    Keyword = state["keyword"]
    Present_agent_generate = state["present_agent_generate"]
    Moderator_feedback = state["moderator_feedback"]
    Meaningless_3_2_count = state["meaningless_3_2_count"]
    Meaning_of_words = state["meaning_of_words"]

    err_count = 0

    ex_experts, ex_re_experts, ex_experts_str = Sep_Agent(Present_agent_generate)

    while True:
        try:
            Re_Result = []
        
            for i, all_experts in enumerate(Experts_Agent, start = 1):
                generate = Re_All_Same_Keyword_Master(Abstract, Keyword, Present_agent_generate, Moderator_feedback, all_experts, Meaning_of_words)
        
                num_result = f'{i}. {generate}'
                Re_Result.append(num_result)
        
            update_agent_generate = '\n\n'.join(Re_Result)

            if err_count > 6:
                break
        
            experts, re_experts, experts_str = Sep_Agent(update_agent_generate)
            break
        except Exception as e:
            print("Sep_Agent error detected → Regenerating")
            err_count += 1
            continue

    compare_keyword_attribute = Compare_Keyword_Attribute(experts, ex_experts)

    if compare_keyword_attribute == False:
        Meaningless_3_2_count += 1

    return GraphState(past_agent_generate = Present_agent_generate, present_agent_generate = update_agent_generate, meaningless_3_2_count = Meaningless_3_2_count)




# 2-2_1. Agent Feedback
def Feedback_Generate(abstract, keyword, my_agent, re_experts_str, meaning_words):
    feedback_prompt = """As an expert in a field, you have a specialized knowledge of your field. Your ultimate goal is to provide feedback on the attribute definitions of other agents, such as entering the role of the agent set for the given field and providing the 'keywords' from the user using the specialized knowledge. In other words, your task is to define the attributes of the keywords by referring to the abstract of the paper provided with the professional agent. When your task analyzes the attributes and reasons defined by other agents using your field's knowledge, if there seems to be an abnormality in the attribute definition value, point it out and refute it, or provide feedback on the attribute definitions of other agents, such as providing feedback on the attribute definitions of other agents, such as providing feedback on the attribute definition value. Agent information and attribute information, abstract of the paper, keywords, attribute definition and reason information for each agent are given, and all answers should be in English."""
    
    feedback_content = """Review the Agent and attribute information below, ensuring compliance with the guidelines. Provide feedback and opinions on the currently defined attribute values.
    
# Agent Info)"""
    
    feedback_content_2 = """
# Feedback Generation Criteria)
1. Assume the role of the assigned Agent among the seven listed, thoroughly analyze the reasoning behind each Agent’s attribute definition, and provide detailed feedback.
2. When defining attributes, the following points must be considered. If any errors related to these aspects are found, the Agent is deemed to have made an incorrect attribute classification, and the feedback should highlight these errors.
 - Agents must prioritize the inherent meaning of the keyword itself. If the keyword is used differently in the abstract than its original meaning, this should be taken into account to ensure the most appropriate attribute classification. 
 - Agent should classify keyword attributes based on their expertise in the assigned field, but not by considering how the keyword is used or perceived within that field. In other words, attribute classification should focus on the inherent meaning of the keyword itself, rather than its value or application in a specific domain.
 - If a keyword is not related to the Agent's assigned field or cannot be clearly classified based on their expertise, the Agent must not define an attribute. Instead, they should mark it with a '-' to indicate that the keyword falls outside their domain and cannot be classified.
 - When defining an attribute, the Agent must provide both the classification and its justification. The justification should be based on the keyword’s meaning and abstract, focusing on why the keyword should be assigned to the chosen attribute. The reasoning must be logically sound and as detailed as possible. Additionally, when providing feedback to other agents, the feedback and its justification must be valid, well-reasoned, and thoroughly explained.
 - Similar keywords may have different attributes depending on their prefixes and suffixes. Therefore, consider both the prefix, suffix, and overall meaning of the keyword when defining its attribute.
 - Each Agent must recall their previously defined attribute value and its justification and use this as a basis for the current task.
 - When classifying attributes, note that observed and examined "phenomena" fall under the Processes & Reactions attribute, whereas observed and measured "results" are classified under Properties & Characteristics. For example, terms such as "Boiling," "Evaporation," "Molecular binding," and "Thermal expansion" belong to Processes & Reactions, while "boiling point," "vapor pressure," "binding energy," and "coefficient of thermal expansion" fall under Properties & Characteristics. Agents must analyze these characteristics to accurately classify the necessary keywords.
3. The assigned Agent must provide feedback on the attribute classifications based on expertise in the relevant field. This includes considering their own previously defined attribute value as well as the justifications provided by other Agents. If any errors or inaccuracies are found in the justifications, they must be pointed out. Conversely, if strong supporting evidence or reasonable justifications exist, they should be presented to help other Agents refine their attribute classifications and ensure accurate classification. Based on the feedback provided, in the subsequent attribute reassessment process, the reviewing Agent may suggest one of three possible actions regarding the attribute classification: 'Modify Attribute Value,' 'Maintain Attribute Value,' or 'Remove Attribute Value.'
 - For example, if an analysis error or incorrect fact is found in another Agent’s justification, it should be pointed out to indicate that the analysis was inaccurate.
 - If another Agent's attribute classification and justification are valid, this should be highlighted to demonstrate the soundness of the reasoning. This allows other Agents to carefully consider the justification when reviewing their attribute classifications.
 - If an analysis error in another Agent's justification is determined to be due to their field being unrelated to the keyword, this should be pointed out. It can then be emphasized that the Agent should not participate in the attribute classification process for that keyword.
 - If, after reviewing the justifications for the currently defined attribute values, it is determined that a more suitable attribute may exist, it should be proposed along with appropriate reasoning. This allows Agents to recognize that, in addition to the initially proposed attribute values, the newly suggested attribute is also a valid classification for the keyword.
 - If an Agent determines that their classified attribute is the most appropriate for the given keyword, they must compare their reasoning with that of other Agents who have assigned different attributes. This comparison should highlight why their classification is more accurate and what key aspects should be considered more carefully. The justification must be well-structured and detailed, providing a strong and logical argument to persuade other Agents who have assigned different attributes.
4. The provided attribute definitions and justifications for each Agent are compiled from those who have contributed meaningful classifications. Any missing Agents have likely not classified the keyword because it is unrelated to their field. Unless there is a special case, these missing Agents should be disregarded, and feedback should be conducted among the existing Agents.
5. Do not generate feedback for Agents that are not included in the provided attribute definitions and justifications unless there is a special case.
6. <Important> The primary focus should be on the inherent meaning of the keyword. However, if the abstract reveals that the keyword is used in a different context from its original meaning, attribute classification should not be based on its association with a specific field ("keyword-field" relationship). Instead, classification must rely on how the keyword is used and interpreted within the abstract ("keyword-abstract" relationship). Therefore, in such cases, the Agent must utilize their domain expertise to analyze how the keyword is described and applied in the abstract. The reasoning for why the keyword should be assigned to a specific attribute must be thoroughly explained and logically justified in the attribute definition. Keep this in mind when providing feedback on the attribute classifications of other Agents, ensuring that sufficient justification is provided.
7. Since seven Agents must deliberate to define a single attribute (or at most two) for a keyword, a wide range of opinions cannot be accommodated. Therefore, critically analyze the proposed attribute classifications for the current keyword and provide feedback centered on the most appropriate classification. In other words, there is no need to fully agree with other Agents’ opinions. Instead, compare and evaluate different perspectives, and select the one that leads to a more factual and meaningful outcome as the basis for your feedback.
8. Each Agent must remember their previously defined attribute classification when providing feedback or counterarguments on others' classifications. There should be no contradictions between their critiques of other Agents' classifications and their own prior definitions. In other words, there must be consistency between previous discussions and current feedback, ensuring that no contradictions arise in the Agent's stance.
9. The following provides an example of feedback generation. While it is not mandatory to follow this example exactly, use it as a reference to understand the approach to generating feedback and apply it accordingly.
  ex1. The classification of 'biofilm' as 'Materials & Sample' by the Life Science Expert acknowledges its significance as a research subject. However, in this study, 'biofilm' is not merely confined to being a research material; rather, the focus is on examining its reactions and processes under physical and chemical treatments. Therefore, the 'Processes & Reactions' attribute more comprehensively reflects the essence of the research, as it emphasizes the study of these processes and reactions. Given that the primary objective of this study is to analyze the response of 'biofilm' to specific treatments, it is necessary to consider a more comprehensive classification.
  ex2. The classification of 'biofilm' as 'Applications & Uses' by the Healthcare Expert holds significance in terms of its potential applications. However, the abstract primarily focuses on the treatment process and outcomes of 'biofilm' rather than its direct application to food safety or public health. Given that the experimental aspects are emphasized rather than direct practical applications, the 'Materials & Sample' attribute, which provides information on the primary subject of the experiment, may be more appropriate than 'Applications & Uses.' The Healthcare Expert’s classification should focus more on the inherent characteristics of 'biofilm' and its treatment process as discussed in the study, rather than how the research findings may be applied. Therefore, a revision is necessary.
10. The provided 'Currently Defined Attribute Values and Justification Information' is structured such that for each mentioned attribute value, the reasoning provided by the Agents who defined the keyword under that attribute is listed accordingly.
11. The attribute definition must strictly define the primary attribute of the given keyword, and supplementary attributes should not be provided.
12. The generated output must strictly follow the structure below. Do not include any unnecessary content beyond this format. (The information inside parentheses serves as a guideline and should be replaced with the corresponding content when writing.)
Generation Structure)
(Write the questions, critiques, and feedback regarding the current attribute definitions and the justifications provided by the Agents.)
"""

    keyword_info = f"""# User-Provided Keyword and Related Abstract Info)
- Keyword requiring attribute classification: {keyword}
- Relevant abstract information from the related paper:
{abstract}
- Assigned Agent (The Agent must provide feedback on the attribute values defined by other Agents from their own perspective.) : {my_agent}

# Keyword Meaning (Fully understand the word’s meaning before proceeding with attribute classification))
{meaning_words}

- Currently Defined Attribute Values and Justification Info ([[[]]] within the text refer to the mentioned attribute name, and directly below, the justifications provided by the Agents who defined the attribute are listed.):
{re_experts_str}
"""

    gen_result = ChatModel_Generation(feedback_prompt, feedback_content, keyword_info, feedback_content_2)

    return gen_result


def N_Feedback_Generate(state : GraphState) -> GraphState:
    Abstract = state["abstract"]
    Keyword = state["keyword"]
    Present_agent_generate = state["present_agent_generate"]
    Meaning_of_words = state["meaning_of_words"]

    experts, re_experts, experts_str = Sep_Agent(Present_agent_generate)

    group_result = Group_Experts_By_Non_Topic(re_experts)
    
    feedback_generate = """"""
    

    for expert_name in re_experts.keys():
        sub_feedback_generate = Feedback_Generate(Abstract, Keyword, expert_name, group_result, Meaning_of_words)

        feedback_generate += "Feedback from " + expert_name +")\n"+sub_feedback_generate+"\n\n"

    return GraphState(agent_feedback = feedback_generate)




# 2-2_2. Moderator Feedback 
def Moderator_Feedback_Generate(abstract, keyword, experts_str, meaning_words):
    moderator_feedback_prompt = """As a keyword attribute classification expert, you possess a high level of understanding regarding keywords and attributes, related knowledge, and keyword attribute classification. Your role is to analyze the attributes of keywords defined by experts in various fields and the reasons for their definitions, and to point out and question whether the attribute definitions given by each agent accurately represent the keyword. Information on agents and attribute definitions, as well as specific attribute definitions by each agent, will be provided to you. You will receive abstracts and keyword information from users. All responses should be in English."""

    moderator_feedback_content = """Review the Agent and attribute information below, ensure compliance with the guidelines, and verify whether the attribute values defined by each Agent accurately represent the keyword. Then, follow the instructions provided.

# Agent Info)"""

    moderator_feedback_content_2 = """
# Instructions
1. As a moderator in the discussion, carefully review each Agent's assigned attribute value and the reasoning behind it after the discussion has concluded. Identify any inconsistencies or errors in their own definitions and pose follow-up questions if necessary. If the reasoning is deemed insufficient, request empirical evidence or present new facts to challenge the Agent's attribute assignment. Additionally, offer your own perspective to prompt Agents to reassess their attribute definitions from alternative angles. Steer and mediate the discussion in a way that encourages critical evaluation, ultimately guiding the Agents toward the most well-founded and accurate attribute assignments.
2. In the attribute definitions and rationale provided by each Agent, a '-' has been used to indicate cases where an Agent could not define an attribute for a keyword unrelated to their field. Therefore, unless a definition is explicitly incorrect, the '-' should be maintained to signify the inability to define an attribute.
3. If all Agents have assigned appropriate attribute values to the keyword, there is no need to raise any questions. In this case, the question field should be marked with '-'.
4. Since the keyword attributes are defined through this process, when analyzing each Agent, carefully identify and evaluate any weaknesses in the attribute definitions with a strict and objective perspective.
5. If any of the seven Agents need to revise their attribute definition, raise questions and provide feedback on aspects they may have overlooked, such as specific conditions or concepts, to guide them toward a more accurate attribute definition.
6. Carefully review the attribute values and justifications provided by each Agent. Identify any errors in the keyword attribute definition, and if the justification lacks sufficient reasoning, request additional evidence or empirical support. Ensure that the attribute is defined correctly based on valid reasoning.
7. If a more meaningful attribute value can be derived than the one defined by the Agent, present it as a question for further consideration.
8. When analyzing the attribute definitions assigned by each Agent, the primary consideration should be ensuring that the assigned attribute aligns with the intrinsic meaning of the keyword itself. If the keyword is used differently in the abstract compared to its original meaning, this should be reflected in the attribute assignment. The analysis should focus on whether the assigned attribute is the most appropriate based on this context and whether the reasoning behind the assignment is logically sound. Additionally, any inconsistencies or issues should be identified and questioned accordingly.
9. <Important> The primary focus should be on the inherent meaning of the keyword. However, if the abstract reveals that the keyword is used in a different context from its original meaning, attribute classification should not be based on its association with a specific field ("keyword-field" relationship). Instead, classification must rely on how the keyword is used and interpreted within the abstract ("keyword-abstract" relationship). Therefore, in such cases, the Agent must utilize their domain expertise to analyze how the keyword is described and applied in the abstract. The reasoning for why the keyword should be assigned to a specific attribute must be thoroughly explained and logically justified in the attribute definition. Keep this in mind and support each Agent in finding sufficient evidence to define the keyword attribute and clearly explain the reasoning behind their definition.
10. For the '7. Topic' attribute, a keyword can be classified under this category when it represents the main subject of the study. Since such keywords can also possess other relevant attributes, they are not limited to the 'Topic' classification alone. Therefore, it is important to note that 'Topic' is the only attribute that can be defined alongside another attribute.
11. <Important> Except for "7. Topic," no attribute should have overlapping definitions. Only the most suitable attribute should be selected.
12. When classifying attributes, note that observed and examined "phenomena" fall under the Processes & Reactions attribute, whereas observed and measured "results" are classified under Properties & Characteristics. For example, terms such as "Boiling," "Evaporation," "Molecular binding," and "Thermal expansion" belong to Processes & Reactions, while "boiling point," "vapor pressure," "binding energy," and "coefficient of thermal expansion" fall under Properties & Characteristics. Carefully analyze these characteristics when determining the appropriate attribute for a given keyword.
13. Similar keywords may have different attributes depending on their prefixes and suffixes. Therefore, consider both the prefix, suffix, and overall meaning of the keyword when defining its attribute.
14. Agent should classify keyword attributes based on their expertise in the assigned field, but not by considering how the keyword is used or perceived within that field. In other words, attribute classification should focus on the inherent meaning of the keyword itself, rather than its value or application in a specific domain.
16. When presenting questions and determining the validity of each Agent's attribute definition, carefully review their assigned attribute value and justification. Ensure that the Agent has not reached an incorrect conclusion by including false or misleading information and that their classification is based on factual evidence.
17. If, after reviewing all the justifications for the currently defined attribute values, there is a possibility that a more suitable attribute could be assigned, it is essential to present that attribute along with appropriate reasoning. This allows for highlighting the fact that, in addition to the proposed attributes, the newly suggested attribute is also a valid classification for the keyword.
18. The attribute definition must strictly define the primary attribute of the given keyword, and supplementary attributes should not be provided.
19. The generated output must strictly follow the structure below. Do not include any unnecessary content beyond this format. (The information inside parentheses serves as a guideline and should be replaced with the corresponding content when writing.)
Generation Structure)
Moderator Feedback)
(Document the questions, inquiries, feedback, and the Moderator's opinions regarding the attribute definitions made by Agents, as identified during the concluded discussion process.)
"""

    keyword_info = f"""# Keyword Meaning (Fully understand the word’s meaning before proceeding with attribute classification))
{meaning_words}

# User-Provided Keyword and Related Abstract Info)
- Keyword requiring attribute classification: {keyword}
- Relevant abstract information from the related paper:
{abstract}

# Attribute Definitions and Justifications for Each Agent:
{experts_str}
"""

    gen_result = ChatModel_Generation(moderator_feedback_prompt, moderator_feedback_content, keyword_info, moderator_feedback_content_2)

    return gen_result


def N_Moderator_Feedback_Generate(state : GraphState) -> GraphState:
    Abstract = state["abstract"]
    Keyword = state["keyword"]
    Present_agent_generate = state["present_agent_generate"]
    Meaning_of_words = state["meaning_of_words"]

    
    experts, re_experts, experts_str = Sep_Agent(Present_agent_generate)

    group_result = Group_Experts_By_Non_Topic(re_experts)

    moderator_feedback_generate = Moderator_Feedback_Generate(Abstract, Keyword, group_result, Meaning_of_words)

    
    return GraphState(moderator_feedback = moderator_feedback_generate)




# 3-3. Revised Attribute Definition
def Reflect_Feedback_Generate(abstract, keyword, my_agent, experts_str, feedback_generate, moderator_feedback_generate, meaning_words):
    reflect_feedback_prompt = """As an expert in your field, you possess specialized knowledge. Your role is to assume the persona of an Agent within your designated field and use your expertise to classify keywords based on the provided abstract of a research paper. Specifically, your task is to define the attributes of the keywords according to the given domain, considering the feedback on keyword attribute definitions from other Agents. You will be provided with information about the Agent, the attributes, the research abstract, keywords, and the feedback on keyword attribute definitions from each Agent. All responses should be in English."""

    reflect_feedback_content = """Review the following Agent and attribute information and adhere to the given guidelines. Assume the role of the assigned Agent and utilize your expertise in the relevant field to revise or remove your previously assigned attribute value based on the feedback provided by other Agents.

# Agent Info)"""

    reflect_feedback_content_2 = """
# Generation Criteria)
1. Assume the role of the assigned Agent among the seven provided Agents and utilize your expertise in the relevant field to either maintain or revise your attribute definition based on the feedback from other Agents and the opinions of the discussion moderator (Moderator).
2. When reviewing their attribute definitions, Agents can choose from the following options:
 - If an Agent finds that others share the same opinion about their attribute definition and the feedback provided is valid, they should retain their original attribute definition without modification.
 - If an Agent finds the counterargument against their attribute definition valid, they should modify their attribute definition accordingly and also revise the reasoning behind the definition.
 - If an Agent disagrees with the opposing feedback or believes that their attribute definition is more valid despite the feedback, they should strongly defend their attribute by providing solid reasoning and counterarguments.
3. If an Agent decides to remove the keyword attribute value, they must refrain from further attribute definition and indicate their field’s irrelevance by marking it with '-'.
4. The reasoning for attribute definition must take into account each Agent's feedback and the opinions of the discussion moderator. Additionally, it should be supported by appropriate evidence to justify why the keyword should be classified under the given attribute, ensuring the rationale is well-founded and valid.
5. Agents must prioritize the inherent meaning of the keyword when defining attributes. However, if the abstract indicates that the keyword is used in a different context than its original meaning, this should be taken into account, and the most appropriate attribute should be assigned accordingly. 
6. <Important> The primary focus should be on the inherent meaning of the keyword. However, if the abstract reveals that the keyword is used in a different context from its original meaning, attribute classification should not be based on its association with a specific field ("keyword-field" relationship). Instead, classification must rely on how the keyword is used and interpreted within the abstract ("keyword-abstract" relationship). Therefore, in such cases, the Agent must utilize their domain expertise to analyze how the keyword is described and applied in the abstract. The reasoning for why the keyword should be assigned to a specific attribute must be thoroughly explained and logically justified in the attribute definition. Keep this in mind while redefining the keyword attribute and provide a well-reasoned explanation for the revised classification.
7. Agent should classify keyword attributes based on their expertise in the assigned field, but not by considering how the keyword is used or perceived within that field. In other words, attribute classification should focus on the inherent meaning of the keyword itself, rather than its value or application in a specific domain.
8. For the '7. Topic' attribute, a keyword can be classified under this category when it represents the main subject of the study. Since such keywords can also possess other relevant attributes, they are not limited to the 'Topic' classification alone. Therefore, it is important to note that 'Topic' is the only attribute that can be defined alongside another attribute.
9. When classifying attributes, note that observed and examined "phenomena" fall under the Processes & Reactions attribute, whereas observed and measured "results" are classified under Properties & Characteristics. For example, terms such as "Boiling," "Evaporation," "Molecular binding," and "Thermal expansion" belong to Processes & Reactions, while "boiling point," "vapor pressure," "binding energy," and "coefficient of thermal expansion" fall under Properties & Characteristics. Carefully analyze these characteristics when determining the appropriate attribute for a given keyword.
10. Similar keywords may have different attributes depending on their prefixes and suffixes. Therefore, consider both the prefix, suffix, and overall meaning of the keyword when defining its attribute.
11. In the provided attribute definition feedback information for each Agent, if the attribute definition value is '-', it indicates that the respective Agents did not classify the attribute because the keyword is unrelated to their field. Therefore, ignore these cases and proceed with providing feedback on the existing Agents' attribute definitions.
12. Since the seven Agents must discuss and determine a single attribute (or at most two) for each keyword, a wide range of opinions cannot be accommodated. Therefore, critically analyze both your own attribute definition and those of other Agents, focusing on the most appropriate attribute definition for critique and feedback. If, as a result of this feedback process, another Agent's attribute definition appears to provide a more meaningful classification than your own, remove your attribute definition.
 In other words, it is not necessary to fully agree with another Agent’s opinion. Instead, compare and analyze both your own and others' perspectives, selecting the one that produces a more factual and meaningful result. Then, base your classification on this selection. However, if after reviewing all previously defined attributes and their justifications, you determine that both your attribute and another Agent’s attribute are equally valid, you may define both attributes. In all other cases, only one attribute must be defined.
 Additionally, attribute definitions must focus on the primary attribute for the given keyword. Secondary attributes should not be proposed.
13. Each Agent must remember their previously defined attribute values when making revisions, ensuring that no contradictions arise. While it is acceptable to agree or disagree with another Agent’s opinion, there must be no inconsistencies between past discussions and the current feedback. Additionally, each Agent should verify that their stance remains logically consistent and free from contradictions. 
14. <Important> The justification must not include the Agent's name or any specific field. Instead, it should present a comprehensive reasoning for the keyword’s attribute classification. When describing the justification, avoid references such as "based on the opinions of all Agents" or "as stated by ?? expert." Focus solely on the general basis for the attribute classification.
15. If the Moderator requests specific information or poses questions, ensure that these are addressed when redefining the attribute and providing the reasoning for its definition.
16. The generated output must strictly follow the structure below. No unnecessary content should be included beyond this format, and responses must adhere precisely to the specified structure. (Insert the assigned Agent name in "OOO" and replace "~~~" with the appropriate values.)
Generation Structure) OOO Expert\nKeyword Attribute: ~~~\nJustification: ~~~
"""

    keyword_info = f"""# Keyword Meaning (Fully understand the word’s meaning before proceeding with attribute classification))
{meaning_words}

# User-Provided Keyword and Related Abstract Info)
- Keyword requiring attribute classification: {keyword}
- Relevant abstract information from the related paper:
{abstract}

- Assigned Agent for Attribute Classification: {my_agent}


- Attribute Definitions and Justifications for Each Agent:
{experts_str}


- Attribute Definition Feedback by Each Agent:
{feedback_generate}


- Feedback Information from the Moderator:
{moderator_feedback_generate}
"""

    gen_result = ChatModel_Generation(reflect_feedback_prompt, reflect_feedback_content, keyword_info, reflect_feedback_content_2)

    return gen_result


def N_Reflect_Feedback_Generate(state : GraphState) -> GraphState:
    Abstract = state["abstract"]
    Keyword = state["keyword"]
    Present_agent_generate = state["present_agent_generate"]
    Agent_feedback = state["agent_feedback"]
    Moderator_feedback = state["moderator_feedback"]
    Meaningless_4_2_count = state["meaningless_4_2_count"]
    Meaning_of_words = state["meaning_of_words"]

    err_count = 0

    Debate_state = "Re-discussion"

    ex_experts, ex_re_experts, ex_experts_str = Sep_Agent(Present_agent_generate)

    while True:
        try:
            R_Result = []
            
            
            for i, all_experts in enumerate(Experts_Agent, start = 1):
                generate = Reflect_Feedback_Generate(Abstract, Keyword, all_experts, Present_agent_generate, Agent_feedback, Moderator_feedback, Meaning_of_words)
        
                num_result = f'{i}. {generate}'
                R_Result.append(num_result)
        
            reflect_feedback_generate = '\n\n'.join(R_Result)

            if err_count > 6:
                break
        
            experts, re_experts, experts_str = Sep_Agent(reflect_feedback_generate)
            break
        except Exception as e:
            print("Sep_Agent error detected → Regenerating")
            err_count += 1
            continue

    compare_keyword_attribute = Compare_Keyword_Attribute(experts, ex_experts)


    if compare_keyword_attribute == True:
        attributes_check = list(re_experts.values())

        if len(attributes_check) > 0:
            if Check_Keyword_Attribute_Count(re_experts) == 1:
                Debate_state = "Final_Attribute_Derivation"
        
    if Meaningless_4_2_count > 4:
        Debate_state = "Discussion_Termination"


    
    if compare_keyword_attribute == False:
        Meaningless_4_2_count += 1            

    return GraphState(past_agent_generate = Present_agent_generate, present_agent_generate = reflect_feedback_generate, meaningless_4_2_count = Meaningless_4_2_count,debate_state = Debate_state)




# 5. Ask agents to choose the most appropriate attribute for remaining keywords (final definition)
def Select_Final_Keyword_Attribute(abstract, keyword, present_agent_generate, my_agent, meaning_words):
    select_final_keyword_attribute_prompt = """You are an expert in your field, possessing specialized knowledge. Your role should move into the role of the agent set for the given field and use your expertise to finally select the best attribute for the keyword among the keyword attribute values derived so far by referring to the previously defined keyword attribute definition value for each agent. Agent information, attribute information, and attribute definition value information for each previous agent will be given, and abstract of the paper and keyword information will be received from the user. All answers should be in English."""

    select_final_keyword_attribute_content = """Check the provided Agent and attribute information, and adhere to the guidelines. Assume the role of the assigned Agent and, based on the previous attribute definition discussion and the given abstract, select the most appropriate final attribute for the keyword.

# Agent Info)"""

    select_final_keyword_attribute_content_2 = """
# Agent Configuration and Attribute Definition Criteria)
1. Assume the role of the assigned Agent among the seven provided Agents and utilize your expertise in the relevant field. Refer to the previous discussion information to select and finalize the most appropriate attribute for the given keyword.
2. Each Agent must recall the previously defined attribute value they assigned and use it as a basis to proceed with the following process.
3. <Important> Through this process, the attribute of the keyword is ultimately determined. Therefore, each Agent must refer to the previously defined attribute values (previous discussion information) and select the most appropriate attribute from the attributes derived in the previous discussion so that all Agents converge on the same attribute value. However, if, after reviewing all previous attribute definitions and their justifications, an Agent determines that both their own attribute and another Agent's attribute are equally valid, both attributes may be defined. Otherwise, it is preferable to define only one attribute.
4. If an Agent previously indicated that the keyword is unrelated to their field by using '-', they should continue to mark it as '-' without defining an attribute, consistently indicating that the keyword is outside their area of expertise.
5. When selecting the final attribute, each Agent must provide not only the chosen attribute value but also the reasoning behind its definition.
6. When defining an attribute, provide both the classification and a justification. The reasoning must be based on the keyword’s inherent meaning and the abstract, focusing on why the keyword should be assigned to the chosen attribute. The justification should be as detailed as possible, ensuring logical validity and supporting the classification with strong evidence.
7. <Important> The primary focus should be on the inherent meaning of the keyword. However, if the abstract reveals that the keyword is used in a different context from its original meaning, attribute classification should not be based on its association with a specific field ("keyword-field" relationship). Instead, classification must rely on how the keyword is used and interpreted within the abstract ("keyword-abstract" relationship). Therefore, in such cases, the Agent must utilize their domain expertise to analyze how the keyword is described and applied in the abstract. The reasoning for why the keyword should be assigned to a specific attribute must be thoroughly explained and logically justified in the attribute definition. Keep this in mind while redefining the keyword attribute and provide a well-reasoned explanation for the revised classification.
8. Agent should classify keyword attributes based on their expertise in the assigned field, but not by considering how the keyword is used or perceived within that field. In other words, attribute classification should focus on the inherent meaning of the keyword itself, rather than its value or application in a specific domain.
9. Each of the 1-7 Agents represents a broad category, so sub-field experts should be assigned within each Agent. These sub-field experts must engage in discussions to determine the most appropriate attribute classification, which will then be established as the final attribute classification for that Agent. For example, in the case of the '1. Natural Science Expert' Agent, experts in physics, astronomy, meteorology, chemistry, geology, and geophysics possess extensive knowledge in their respective domains. Therefore, after discussions among these specialists, the most suitable attribute classification should be determined and set as the final classification for the '1. Natural Science Expert' Agent.
10. For the '7. Topic' attribute, a keyword can be classified under this category when it represents the main subject of the study. Since such keywords can also possess other relevant attributes, they are not limited to the 'Topic' classification alone. Therefore, it is important to note that 'Topic' is the only attribute that can be defined alongside another attribute.
11. <Important> Except for "7. Topic," no attribute should have overlapping definitions. Only the most suitable attribute should be selected.
12. When classifying attributes, note that observed and examined "phenomena" fall under the Processes & Reactions attribute, whereas observed and measured "results" are classified under Properties & Characteristics. For example, terms such as "Boiling," "Evaporation," "Molecular binding," and "Thermal expansion" belong to Processes & Reactions, while "boiling point," "vapor pressure," "binding energy," and "coefficient of thermal expansion" fall under Properties & Characteristics. Carefully analyze these characteristics when determining the appropriate attribute for a given keyword.
13. Similar keywords may have different attributes depending on their prefixes and suffixes. Therefore, consider both the prefix, suffix, and overall meaning of the keyword when defining its attribute.
14. Since the seven Agents must discuss and agree on a single attribute for a given keyword, a broad range of opinions cannot be accommodated. Therefore, each Agent should critically analyze both their own attribute definition and those of other Agents to determine and adopt the most appropriate attribute. If feedback suggests that another Agent’s attribute definition is more meaningful and yields a better classification, the Agent should remove their initial attribute definition. However, there is no obligation to fully agree with another Agent’s opinion. Instead, the Agent should compare and analyze their own perspective against those of others and select the most factually accurate and meaningful conclusion as the basis for the final decision.
15. Attribute classification must strictly define the primary attribute of the keyword. Do not provide auxiliary or secondary attributes.
16. The generated output must strictly follow the structure below. No unnecessary content should be included beyond this format, and responses must adhere precisely to the specified structure. (Insert the assigned Agent name in "OOO" and replace "~~~" with the appropriate values.)
Generation Structure) OOO Expert\nKeyword Attribute: ~~~\nJustification: ~~~
"""

    keyword_info = f"""# Keyword Meaning (Fully understand the word’s meaning before proceeding with attribute classification))
{meaning_words}

# User-Provided Keyword and Related Abstract Info)
- Keyword requiring attribute classification: {keyword}
- Relevant abstract information from the related paper:
{abstract}


- Assigned Agent for Attribute Classification : {my_agent}


# Each Agent's Previously Defined Attribute and Justification (Previous Discussion Information))
{present_agent_generate}
"""

    gen_result = ChatModel_Generation(select_final_keyword_attribute_prompt, select_final_keyword_attribute_content, keyword_info, select_final_keyword_attribute_content_2)

    return gen_result


def N_Select_Final_Keyword_Attribute(state : GraphState) -> GraphState:
    Abstract = state["abstract"]
    Keyword = state["keyword"]
    Present_agent_generate = state["present_agent_generate"]
    Meaning_of_words = state["meaning_of_words"]

    experts, re_experts, experts_str = Sep_Agent(Present_agent_generate)
    
    F_Result = []

    for expert_name in re_experts:
        generate = Select_Final_Keyword_Attribute(Abstract, Keyword, Present_agent_generate, expert_name, Meaning_of_words)

        generate_result = f'- {generate}'
        F_Result.append(generate_result)

    update_agent_generate = '\n\n'.join(F_Result)


    return GraphState(past_agent_generate = Present_agent_generate, present_agent_generate = update_agent_generate)




# 6. Final attribute definition(Consider multiple definitions) 
def Final_Step_Multi_Keyword_Attribute(abstract, keyword, present_agent_generate):
    final_step_Multi_prompt = """You are a helpful assistant. Your task is to gather the opinions of each expert and give them appropriate attributes for the 'keywords' they get from the user, and summarize the reasons for defining the keyword attributes by referring to the attribute definitions defined by experts in each field. In other words, you must combine the attribute definitions defined by experts in each field with their expertise in their field, and provide them with a summary of the attribute definitions for the keyword and why. Agent information and attribute information, the attribute definition values for each agent, and the reason information will be given, and the abstract of the thesis and keyword information will be received from the user. All answers should be in English."""

    final_step_Multi_content = """Review the following Agent and attribute information, and ensure compliance with the provided guidelines. Consolidate the attribute values and justifications defined by each Agent into a single, coherent statement to derive the final attribute for the given keyword.

# Agent Info)"""

    final_step_Multi_content_2 = """
# Agent Configuration and Attribute Definition Criteria)
1. If an Agent's assigned keyword attribute value is '-', it indicates that the keyword is unrelated to their field and should be excluded. Instead, refer to the meaningful attribute values provided by the relevant Agents.
2. Since the final attribute for the keyword must be determined, consolidate the attribute values and justifications provided by each Agent into a clear and structured summary.
3. When defining an attribute, provide both the classification and a justification. The reasoning must be based on the keyword’s inherent meaning and the abstract, focusing on why the keyword should be assigned to the chosen attribute. The justification should be as detailed as possible, ensuring logical validity and supporting the classification with strong evidence.
4. For the '7. Topic' attribute, a keyword can be classified under this category when it represents the main subject of the study. Since such keywords can also possess other relevant attributes, they are not limited to the 'Topic' classification alone. Therefore, it is important to note that 'Topic' is the only attribute that can be defined alongside another attribute.
5. <Important> Attributes other than '7. Topic' cannot be defined redundantly, and only the most appropriate attribute should be selected. However, if, after reviewing the attribute values and justifications provided by each Agent, an Agent determines that another Agent's attribute is also highly valid in addition to their own, both attributes may be defined. In all other cases, only one attribute must be chosen.
6. When classifying attributes, note that observed and examined "phenomena" fall under the Processes & Reactions attribute, whereas observed and measured "results" are classified under Properties & Characteristics. For example, terms such as "Boiling," "Evaporation," "Molecular binding," and "Thermal expansion" belong to Processes & Reactions, while "boiling point," "vapor pressure," "binding energy," and "coefficient of thermal expansion" fall under Properties & Characteristics. Carefully analyze these characteristics when determining the appropriate attribute for a given keyword.
7. Similar keywords may have different attributes depending on their prefixes and suffixes. Therefore, consider both the prefix, suffix, and overall meaning of the keyword when defining its attribute.
8. If the attribute values defined by each Agent are not identical, the final attribute definition should not include both values. Instead, the most appropriate attribute should be selected based on the reasoning provided by each Agent or determined by majority consensus, ensuring that only one attribute is defined as the final value. However, if the attribute values and justifications provided by each Agent are equally valid and it is impossible to define a single attribute, two attribute values may be defined. (Exception: The "Topic" attribute can be selected in multiple instances.)
9. <Important> The justification must not include the Agent's name or any specific field. Instead, it should present a comprehensive reasoning for the keyword’s attribute classification. When describing the justification, avoid references such as "based on the opinions of all Agents" or "as stated by ?? expert." Focus solely on the general basis for the attribute classification.
10. The generated output must follow the structure "Keyword : Attribute Type - Reason." Do not include any unnecessary content beyond this format.
In other words, the keyword name and its defined attribute value should be separated by a colon (:), and the attribute value and its justification should be separated by a hyphen (-).
"""

    keyword_info = f"""# User-Provided Keyword and Related Abstract Info)
- Keyword requiring attribute classification: {keyword}
- Relevant abstract information from the related paper:
{abstract}

# Attribute Definitions and Justifications for Each Agent)
{present_agent_generate}


"""

    gen_result = ChatModel_Generation(final_step_Multi_prompt, final_step_Multi_content, keyword_info, final_step_Multi_content_2)

    return gen_result

def N_Final_Step_Multi_Keyword_Attribute(state : GraphState) -> GraphState:
    Abstract = state["abstract"]
    Keyword = state["keyword"]
    present_agent_generate = state["present_agent_generate"]
    
    
    final_keyword_result = Final_Step_Multi_Keyword_Attribute(Abstract, Keyword, present_agent_generate)

    return GraphState(Final_Keyword_Result = final_keyword_result)










def NE_check_attribute_being(state: GraphState) -> GraphState:
    present_agent_generate = state["present_agent_generate"]

    Experts, Re_experts, Experts_str = Sep_Agent(present_agent_generate)

    attributes = list(Re_experts.values())

    if len(attributes) > 0:
        check_keyword_attribute_count = Check_Keyword_Attribute_Count(Experts)

        if check_keyword_attribute_count == 1:
            statue = "Identical_Attribute_Definition"
        else:
            statue = "Multiple_Attribute_Definition"
    else:
        statue = "err"

    return GraphState(status=statue)

def NE_check_definability(state: GraphState) -> GraphState:
    return state["definability"]
        

def NE_meaningless_3_2_count_check(state: GraphState) -> GraphState:
    Meaningless_3_2_count = state["meaningless_3_2_count"]

    if Meaningless_3_2_count > 4:
        Meaningless_3_2_definition = "Discussion_Termination"
    else:
        Meaningless_3_2_definition = "Continue_Discussion"

    return GraphState(meaningless_3_2_definition = Meaningless_3_2_definition)

def NE_meaningless_3_2_next_step(state: GraphState) -> GraphState:
    return state["meaningless_3_2_definition"]


def NE_debate_next_step(state: GraphState) -> GraphState:
    return state["debate_state"]


def NE_next_step(state: GraphState) -> GraphState:
    return state["status"]





#################################################################################################################################################
from langgraph.graph import END, START, StateGraph
from langgraph.checkpoint.memory import MemorySaver


def Workflow() -> StateGraph:
    
    workflow = StateGraph(GraphState)
    

    workflow.add_node("N_Meaning_Of_Keyword", N_Meaning_Of_Keyword)
    workflow.add_node("N_All_First_Same_Agent_Generate", N_All_First_Same_Agent_Generate)
    workflow.add_node("N_All_Same_Keyword_Master", N_All_Same_Keyword_Master)
    workflow.add_node("N_Final_Step_Keyword_Attribute", N_Final_Step_Keyword_Attribute)
    workflow.add_node("NE_check_attribute_being", NE_check_attribute_being)
    workflow.add_node("N_Re_All_Same_Keyword_Master", N_Re_All_Same_Keyword_Master)
    workflow.add_node("NE_meaningless_3_2_count_check", NE_meaningless_3_2_count_check)
    workflow.add_node("N_Feedback_Generate", N_Feedback_Generate)
    workflow.add_node("N_Moderator_Feedback_Generate", N_Moderator_Feedback_Generate)
    workflow.add_node("N_Reflect_Feedback_Generate", N_Reflect_Feedback_Generate)
    workflow.add_node("N_Select_Final_Keyword_Attribute", N_Select_Final_Keyword_Attribute)
    workflow.add_node("N_Final_Step_Multi_Keyword_Attribute", N_Final_Step_Multi_Keyword_Attribute)
    
    
    workflow.add_edge("N_Meaning_Of_Keyword", "N_All_First_Same_Agent_Generate")
    workflow.add_edge("N_All_First_Same_Agent_Generate", "NE_check_attribute_being")
    
    workflow.add_conditional_edges(
        "NE_check_attribute_being",
        NE_next_step,
        {
            "Identical_Attribute_Definition" : "N_All_Same_Keyword_Master",
            "Multiple_Attribute_Definition" : "N_Feedback_Generate",
            "err" : "N_All_First_Same_Agent_Generate",
        },
    )
    
    workflow.add_conditional_edges(
        "N_All_Same_Keyword_Master",
        NE_check_definability,
        {
            "possible" : "N_Final_Step_Keyword_Attribute",
            "not possible" : "N_Re_All_Same_Keyword_Master",
            "err" : "N_All_Same_Keyword_Master", 
        },
    )
    
    workflow.add_edge("N_Re_All_Same_Keyword_Master", "NE_meaningless_3_2_count_check")
    
    workflow.add_conditional_edges(
        "NE_meaningless_3_2_count_check",
        NE_meaningless_3_2_next_step,
        {
            "Discussion_Termination" : "N_Select_Final_Keyword_Attribute",
            "Continue_Discussion" : "NE_check_attribute_being",
        },
    )
    
    workflow.add_edge("N_Feedback_Generate", "N_Moderator_Feedback_Generate")
    workflow.add_edge("N_Moderator_Feedback_Generate", "N_Reflect_Feedback_Generate")
    
    workflow.add_conditional_edges(
        "N_Reflect_Feedback_Generate",
        NE_debate_next_step,
        {
            "Final_Attribute_Derivation" : "N_Final_Step_Keyword_Attribute",
            "Re-discussion" : "N_Feedback_Generate",
            "Discussion_Termination" : "N_Select_Final_Keyword_Attribute",
        },
    )
    
    workflow.add_edge("N_Select_Final_Keyword_Attribute", "N_Final_Step_Multi_Keyword_Attribute")
    
    
    
    workflow.add_edge("N_Final_Step_Keyword_Attribute", END)
    workflow.add_edge("N_Final_Step_Multi_Keyword_Attribute", END)
    
    
    workflow.set_entry_point("N_Meaning_Of_Keyword")

    return workflow

In [13]:
from langgraph.errors import GraphRecursionError
from langchain_core.runnables import RunnableConfig


def main(s, e):
    workflow = Workflow()
    

    checkpointer = MemorySaver()
    

    app = workflow.compile(checkpointer=checkpointer)

    ##################################################################################################################
    api_result_test = []
    
    for i in range(s,e):
        user_input_attribute = abstract[i]
        user_input_keyword = keyword[i]
        
        config = RunnableConfig(recursion_limit=100, configurable={"thread_id": "Keyword_Attribute_Debate"})
    
        inputs = GraphState(abstract = user_input_attribute, keyword = user_input_keyword)
    
        try:
            result = {"Abstract": user_input_attribute, "Keyword": user_input_keyword}
    
            step_num = 1
            
            for output in app.stream(inputs, config=config):
                print(f"Abstract : {user_input_attribute}\nKeyword : {user_input_keyword}\n\n")
                
                for key, value in output.items():
                    print("-"*140, "\n")
                    print(f"[[[[ NODE({key}) ]]]]")
                    for k, v in value.items():
                        print(f"<{k}>\n {v}")
    
                        if k not in {"past_agent_generate", "meaningless_3_2_count", "meaningless_4_2_count"}:
                            step_number = f"Step {step_num}"
                            result[step_number] = v
                            step_num += 1
                    print("\n", "-"*140, "\n\n")
            api_result_test.append(result)
            print("\n"*5, "="*130, "\n"*5)
        except GraphRecursionError as e:
            print(f"Recursion limit reached: {e}")
    ##################################################################################################################


if __name__ == "__main__":
    # Setting the scope of keyword classification
    main(0,1)

Abstract : This study explored the impact of gas treatments on the structures of multi-walled carbon nanotubes supported Pd (CNT-Pd) catalysts used for electrocatalytic H2O2 reduction and the Heck cross-coupling reaction. The CNT-Pd catalyst was prepared by anchoring Pd nanoparticles on thiolated CNTs. XPS was conducted to examine the surface composition and electronic structure changes of the CNT-Pd catalyst before and after gas treatment. The XPS results revealed that as-prepared CNT-Pd contains at least two different oxidation states on the surface, whereon their proportions depend on the gas used for treatment. Treatment with Hz leads to Pd(0) enrichment near the surface, while Oz treatment causes Pd(II) enrichment of CNT-Pd. All catalysts containing both Pd(0) and Pd(II) were active toward H2O2 reduction, and the Heck cross-coupling reaction of n-butyl acrylate and 4-iodo- toluene; increased proportion of metallic Pd(0) boosted the catalytic reaction. However, the catalyst stabili